# Full Idempotent Setup

## 1. Setup

In [0]:
dbutils.widgets.text(
    "catalog_name", 
    "apex_bank_demo", 
    "1. Catalog Name"
)
dbutils.widgets.dropdown(
    "reset_mode", 
    "False", 
    ["True", "False"], 
    "2. Nuke & Reset?"
)
catalog_name = dbutils.widgets.get("catalog_name")
reset_mode = dbutils.widgets.get("reset_mode")

## 2. Cleanup

In [0]:
if reset_mode == "True":
    print(f"Dropping catalog {catalog_name}...")
    spark.sql(
        f"DROP CATALOG IF EXISTS {catalog_name} CASCADE"
    )

## 3. Infrastructure

In [0]:
print(f"Building architecture in {catalog_name}...")
# Create Catalog
spark.sql(
    f"CREATE CATALOG IF NOT EXISTS {catalog_name}"
)
spark.sql(
    f"USE CATALOG {catalog_name}"
)
# Create Schemas
spark.sql(
    "CREATE SCHEMA IF NOT EXISTS raw_data"
)
spark.sql(
    "CREATE SCHEMA IF NOT EXISTS analytics"
)
# Create Volume (The Landing Zone)
spark.sql(
    "CREATE VOLUME IF NOT EXISTS raw_data.landing_zone"
)
print("Infrastructure ready")

Building architecture in apex_bank_demo...
Infrastructure ready


## 4. Seed Data

_Note: this is blocked on a shared cluster restricting access to local filesystem_

In [0]:
import os

# Dynamic path retrieval for the Git repo
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
repo_root = f"/Workspace{os.path.dirname(notebook_path)}"
source_path = os.path.join(repo_root, "data")

# Target Volume Path (trailing slash required)
destination_volume_path = (
    f"/Volumes/{catalog_name}/raw_data/landing_zone/"
)

print(
    f"Copying data...\nFrom: {source_path}\nTo: {destination_volume_path}"
)

if not os.path.exists(source_path):
    print(f"Error: Data directory not found at {source_path}")
else:
    files = [
        f for f in os.listdir(source_path) 
        if f.endswith(".csv")
    ]
    for file in files:
        source_file_uri = f"file:{os.path.join(source_path, file)}"
        target_file_uri = f"{destination_volume_path}{file}"
        dbutils.fs.cp(
            source_file_uri, 
            target_file_uri
        )
        print(f"  -> Copied {file}")

print(
    f"Data seeding complete. {len(files)} files transferred."
)

Copying data...
From: /Workspace/Users/philip@thephilipjones.com/apex-bank-demo/data
To: /Volumes/apex_bank_demo/raw_data/landing_zone/


---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-7095524988749325>, line 27
     25         source_file_uri = f"file:{os.path.join(source_path, file)}"
     26         target_file_uri = f"{destination_volume_path}{file}"
---> 27         dbutils.fs.cp(
     28             source_file_uri, 
     29             target_file_uri
     30         )
     31         print(f"  -> Copied {file}")
     33 print(
     34     f"Data seeding complete. {len(files)} files transferred."
     35 )

File /databricks/python_shell/dbruntime/remotefshandler/RemoteFsHandler.py:52, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
     49 class ExecutionError(Exception):
     50     pass
---> 52 raise ExecutionError(str(e)) from None

ExecutionError: (java.lang.SecurityException) Cannot use com.databricks.backend.daemon.driver.WorkspaceLocalFileSystem - local file

## 5. Verify

In [0]:
display(
    dbutils.fs.ls(destination_volume_path)
)

path,name,size,modificationTime
dbfs:/Volumes/apex_bank_demo/raw_data/landing_zone/synthetic_accounts.csv,synthetic_accounts.csv,961096,1763478289000
dbfs:/Volumes/apex_bank_demo/raw_data/landing_zone/synthetic_fraud_labels.csv,synthetic_fraud_labels.csv,8812501,1763478290000
dbfs:/Volumes/apex_bank_demo/raw_data/landing_zone/synthetic_transactions.csv,synthetic_transactions.csv,11383193,1763478290000
